In [11]:
%pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression

In [4]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [37]:
train_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,target
0,52.843,66.492,57.300,25.224,39.213,56.593,27.157,67.453,55.690,36.504,64.123,46.375,78.489,36.789,55.797,43.135,37.513,64.094,43.039,42.852,71.971,54.200,50.865,63.450,55.564,40.377,66.034,27.200,35.914,45.797,59.577,47.491,53.825,29.775,24.519,50.074,55.078,57.136,31.947,29.835,78.451,21.561,65.484,47.571,42.559,42.302,47.089,57.277,46.133,40.266,148.76994
1,50.478,53.147,46.412,38.045,52.160,50.398,60.157,44.022,48.668,45.512,54.922,61.904,46.994,35.492,55.057,43.859,54.722,51.982,43.310,46.613,27.654,34.706,66.246,30.508,56.800,39.621,64.157,63.566,47.243,45.964,44.730,48.376,30.482,40.231,76.050,49.982,65.754,57.731,49.041,62.655,26.166,65.698,62.031,59.613,45.587,59.785,51.325,47.258,46.820,40.191,152.01250
2,16.432,30.201,43.268,76.786,58.864,24.046,46.519,63.948,50.190,65.539,52.754,27.011,26.714,66.842,32.628,47.996,75.920,24.627,56.853,65.180,52.493,52.075,39.979,52.382,51.602,44.463,33.449,65.575,41.230,59.523,66.621,18.551,67.621,57.615,60.326,40.396,51.721,52.085,34.213,53.450,68.084,48.324,37.271,19.784,42.810,43.532,45.287,57.780,67.637,59.250,141.96694
3,39.355,33.501,76.737,57.453,58.991,49.938,42.713,71.836,47.394,64.772,15.456,44.349,45.166,85.341,32.967,26.003,40.192,41.518,74.502,37.807,76.032,55.240,44.643,51.042,32.594,53.732,35.457,53.337,58.401,75.910,60.179,53.020,74.436,80.335,47.937,59.607,43.176,43.737,51.951,53.816,43.235,54.231,16.994,31.166,66.766,57.910,78.467,31.459,39.186,75.009,152.49692
4,30.337,62.905,70.896,35.969,45.878,56.114,45.075,61.684,58.865,41.360,59.412,45.491,65.265,48.155,49.756,48.782,44.316,43.562,41.068,52.805,63.313,41.587,49.712,46.263,66.798,43.327,57.003,42.319,31.023,61.477,41.926,46.877,47.548,57.963,38.951,67.351,46.065,35.630,33.074,29.128,59.715,52.879,40.566,35.881,59.963,40.364,56.930,48.503,50.755,49.024,149.01620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11994,55.125,40.855,51.642,53.804,41.503,33.726,52.772,65.948,39.001,48.631,46.168,55.202,27.775,54.346,52.956,26.134,38.966,55.888,79.577,40.278,42.112,54.795,61.249,24.643,49.410,63.472,64.361,44.259,48.784,48.816,31.499,50.210,52.668,44.321,55.909,47.816,53.332,70.780,62.500,65.148,25.957,47.331,44.780,47.508,63.349,47.741,54.489,54.925,46.501,52.830,153.85584
11995,37.690,48.136,39.153,40.591,65.400,51.964,51.693,69.253,71.744,34.553,61.892,36.747,46.802,58.285,52.654,36.079,39.429,49.476,60.627,43.063,70.211,62.299,55.476,43.439,56.435,44.171,61.864,27.117,19.447,47.264,39.849,42.610,57.119,51.309,19.365,59.312,39.603,57.839,48.755,35.465,54.459,49.043,31.514,31.367,66.361,39.992,61.069,51.600,54.289,56.318,139.71284
11996,43.906,56.481,61.781,27.835,58.112,47.535,52.530,80.888,40.617,45.332,54.239,44.533,63.695,58.676,46.446,39.853,40.191,52.959,60.685,46.609,62.180,61.552,37.036,44.029,51.588,45.573,55.326,33.131,44.117,49.809,38.957,56.255,60.791,50.698,31.548,54.106,52.697,52.097,40.866,43.298,63.292,52.619,30.001,34.233,64.683,52.976,53.807,42.545,60.075,43.807,158.45490
11997,37.697,33.088,39.197,54.305,48.470,25.418,66.033,62.126,54.841,41.474,62.388,57.533,36.299,69.937,46.893,32.901,49.941,37.553,69.896,36.299,65.598,57.305,58.251,46.678,52.761,60.379,61.004,40.289,44.671,52.031,51.572,56.681,65.798,43.639,38.178,53.660,35.868,67.491,61.383,50.254,47.246,64.683,41.975,35.830,60.999,47.417,70.069,48.101,42.074,56.213,146.29474


In [34]:
X_train=train_df.drop(columns=['target'])
y_train=train_df[['target']].values.ravel()

poly1=PolynomialFeatures(interaction_only=True,include_bias=False)
poly2=PolynomialFeatures(interaction_only=False,include_bias=False)
X_train_poly1=poly1.fit_transform(X_train)
X_train_poly2=poly2.fit_transform(X_train)

In [22]:
X_spl1,X_val1,y_spl1,y_val1= train_test_split(X_train_poly1,y_train,test_size=0.2,random_state=42)
model1=RandomForestRegressor(n_estimators=350,verbose=1,random_state=42)
model1.fit(X_spl1,y_spl1)

y_pred1=model1.predict(X_val1)

X_spl2,X_val2,y_spl2,y_val2= train_test_split(X_train_poly2,y_train,test_size=0.2,random_state=42)
model2=RandomForestRegressor(n_estimators=350,verbose=1,random_state=42)
model2.fit(X_spl2,y_spl2)

y_pred2=model2.predict(X_val2)

r_1=r2_score(y_val1,y_pred1)
r_2=r2_score(y_val2,y_pred2)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  9.8min
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed: 39.8min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed: 10.2min
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed: 41.7min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s


In [26]:
r_1

0.5117171634650981

In [27]:
r_2

0.5113676578586112

In [36]:
X_test=test_df.drop(columns=['id'])
X_test
X_test_poly=poly1.transform(X_test)

y_test=model1.predict(X_test_poly)
y_test

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.5s


array([151.3043112 , 147.54572286, 149.73320326, ..., 152.85849006,
       154.50537777, 151.13465983], shape=(12000,))

In [38]:
id_lis=[x for x in test_df['id']]

target_lis=list(y_test)
columns={'id':id_lis,'target':target_lis}

final_df=pd.DataFrame(columns)
final_df

,id,target
0,1,151.304311
1,2,147.545723
2,3,149.733203
3,4,145.935260
4,5,152.068301
...,...,...
11995,11996,160.638947
11996,11997,154.406990
11997,11998,152.858490
11998,11999,154.505378


In [40]:
final_df.to_csv('hack2_predictions2.csv',index=False)